In [6]:
import os
import json
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

bucket_name = "stuartlab"
project_name = "tabula-muris"  # Dataset folder and output location

os.chdir(os.path.expanduser("~/data/tabula-muris"))

In [7]:
import boto3
session = boto3.session.Session(profile_name=os.getenv("AWS_PROFILE"))
bucket = session.resource(
    "s3", endpoint_url=os.getenv("AWS_S3_ENDPOINT")).Bucket(bucket_name)
print("S3 Profile: {} Endpoint: {} Project: {}".format(
    os.getenv("AWS_PROFILE"), os.getenv("AWS_S3_ENDPOINT"), project_name))

metadata = json.loads(bucket.Object(
    project_name + "/FACS/metadata.json").get()['Body'].read().decode('utf-8'))
print("Dataset metadata keys:", list(metadata.keys()))

S3 Profile: prp Endpoint: https://s3.nautilus.optiputer.net Project: tabula-muris
Dataset metadata keys: ['num_test_samples', 'genes', 'num_train_samples', 'tissues']


In [8]:
params = json.loads(bucket.Object(
    "{0}/models/{0}.params.json".format(project_name)).get()['Body'].read().decode('utf-8'))

import tempfile
tempname = next(tempfile._get_candidate_names())

bucket.Object("{0}/models/{0}.model.h5".format(project_name)).upload_file(
    "/tmp/{}.h5".format(tempname), ExtraArgs={"ACL":"public-read"})


model.save("/tmp/{}.h5".format(tempname))
bucket.Object("{0}/models/{0}.model.h5".format(project_name)).upload_file(
    "/tmp/{}.h5".format(tempname), ExtraArgs={"ACL":"public-read"})
os.remove("/tmp/{}.h5".format(tempname))

In [8]:
model = tf.keras.models.model_from_json(open("model/tabula-muris.model.json").read())
model.load_weights("model/tabula-muris.weights.h5")